In [2]:
import gensim
import pandas as pd
import json

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)  

In [ ]:
model

In [ ]:
import json

with open('./src/datasets.json','r') as f:
    datasets = json.load(f)
    
# datasets=datasets['results']

In [ ]:
len(datasets)

In [ ]:
lookup ={
}
rlookup={
}
def read_corpus(datasets, tokens_only=False):
    for index,dataset in enumerate(datasets):
            tokens = gensim.utils.simple_preprocess(dataset['resource']['name'] + " " + dataset['resource']['description'])
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                lookup[dataset['resource']['id']] = index
                rlookup[index] = dataset['resource']['id']
                yield gensim.models.doc2vec.TaggedDocument(tokens, [index])

train_corpus = list(read_corpus(datasets))
# test_corpus = list(read_corpus(lee_test_file, tokens_only=True))



In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [134]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [135]:
vector = model.infer_vector(train_corpus[0].words)
vector

array([ 0.24113488,  1.4916537 ,  0.7330141 , -0.85479075, -0.46601725,
        0.284504  ,  0.48587903, -1.67684   , -0.5447132 ,  0.07716218,
        0.24809162, -0.60581654, -0.06841498,  0.25660345,  0.93125534,
        0.8796326 , -1.1321143 , -0.6407086 ,  0.14011076,  0.21565337,
       -0.27137345, -0.58206457,  0.38225177,  0.31509495,  0.14357023,
        0.09116443, -0.46132284,  0.20667283, -0.5688424 , -0.3601245 ,
        0.66578853, -0.36837518, -0.50423753,  0.51635844,  0.00209239,
        0.10683946, -0.37305444, -0.13016497, -0.17788894,  0.1438394 ,
        0.15878563,  0.86166734, -0.5937143 , -0.03547985,  0.4861874 ,
       -1.4343865 , -0.18082707,  1.5437032 ,  0.02547218, -0.22775139],
      dtype=float32)

In [136]:
datasets[0]['resource']['id']

'fuzi-xuv4'

In [137]:
docForId('ic3t-wcy2')

IndexError: list index out of range

In [158]:
a = [23,4,5]
b = [234,5]
a.extend(b)
a


[23, 4, 5, 234, 5]

In [ ]:
results = {}
for datasetid in lookup.keys():
    inferred_vector = model.infer_vector(train_corpus[lookup[datasetid]].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=30)
    sims = [ {'dataset': rlookup[sim[0]], 'similarity': sim[1]} for sim in sims]
    results[datasetid] = sims

In [ ]:
len(results.keys())

In [ ]:
with open('public/similarity_metrics.json', 'w') as f:
    json.dump(results,f)

In [124]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [126]:
results['jign-uhe6']

[{'dataset': 'jign-uhe6', 'similarity': 0.6124996542930603},
 {'dataset': 'shpd-5q9m', 'similarity': 0.5990572571754456},
 {'dataset': 'i6bk-bwyv', 'similarity': 0.5953792333602905},
 {'dataset': 'i4kb-6ab6', 'similarity': 0.590259313583374},
 {'dataset': 'rgy2-tti8', 'similarity': 0.587500274181366},
 {'dataset': '2mhq-um7h', 'similarity': 0.5857139825820923},
 {'dataset': 'fx4z-5xg2', 'similarity': 0.5824402570724487},
 {'dataset': '799n-b76v', 'similarity': 0.5821795463562012},
 {'dataset': '9z9b-6hvk', 'similarity': 0.5805541276931763},
 {'dataset': 'c2g8-ercv', 'similarity': 0.579156756401062}]

In [125]:
results['mnz3-dyi8']

[{'dataset': 'jign-uhe6', 'similarity': 0.6124996542930603},
 {'dataset': 'shpd-5q9m', 'similarity': 0.5990572571754456},
 {'dataset': 'i6bk-bwyv', 'similarity': 0.5953792333602905},
 {'dataset': 'i4kb-6ab6', 'similarity': 0.590259313583374},
 {'dataset': 'rgy2-tti8', 'similarity': 0.587500274181366},
 {'dataset': '2mhq-um7h', 'similarity': 0.5857139825820923},
 {'dataset': 'fx4z-5xg2', 'similarity': 0.5824402570724487},
 {'dataset': '799n-b76v', 'similarity': 0.5821795463562012},
 {'dataset': '9z9b-6hvk', 'similarity': 0.5805541276931763},
 {'dataset': 'c2g8-ercv', 'similarity': 0.579156756401062}]

In [105]:
len(datasets)

100

## All Domains

In [138]:
from urllib.request import urlretrieve

In [139]:
import requests

In [147]:
domains = requests.get("http://api.us.socrata.com/api/catalog/v1/domains").json()['results']

In [160]:

def get_dataset_list_chunk(domain, page,chunk_size):
    offset= page*chunk_size
    limit = chunk_size
    url='https://api.us.socrata.com/api/catalog/v1?domains={domain}&search_context={domain}&offset={offset}&limit={limit}'
    return requests.get(url.format(domain=domain, offset=offset,limit=limit)).json()['results']

In [ ]:
chunk_size=100

results={}
for domain in domains:
    datasets = []
    print('doing ',domain['domain'])
    total = domain['count']
    for i in range(total//chunk_size +1): 
        datasets.extend(get_dataset_list_chunk(domain['domain'], 0 ,100))
    results[domain['domain']]=datasets

In [166]:
with open('all_portal_datasets.json', 'w') as f:
    json.dump(results,f)

In [3]:
with open('all_portal_datasets.json') as f:
    results = json.load(f)

## All dataset document modeling

In [4]:
all_data_lookup={}
reverse_all_data_lookup={}
index = 0

def read_corpus():
    for domain in results.keys():
        for dataset in results[domain]:
            dataset_id = dataset['resource']['id']
            key = "{domain}_{did}".format(domain=domain, did=dataset_id)
            all_data_lookup[key] = index 
            reverse_all_data_lookup[index] = key
            description = dataset['resource']['description']
            tokens = gensim.utils.simple_preprocess(dataset['resource']['name'] + " " + dataset['resource']['description'])
            yield gensim.models.doc2vec.TaggedDocument(tokens, [index])
train_corpus = list(read_corpus())


In [ ]:
all_model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
all_model.build_vocab(train_corpus)
all_model.train(train_corpus, total_examples=all_model.corpus_count, epochs=all_model.epochs)

## 